# Part II - VacationPyVacationPy
----
Use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

* Create a heat map that displays the humidity for every city from Part I.

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.
* If you having trouble displaying the maps, try running `jupyter nbextension enable --py gmaps` in your environment and retry.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests as rq
import gmaps
import os
from pprint import pprint
# Import API key
from config import google_api

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
cities_df = pd.read_csv('../output_data/cities2.csv')
cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,1645133,Dumai,100,ID,1612062421,54,1.6833,101.4500,85.30,3.42
1,3579925,St. George's,20,GD,1612062421,83,12.0564,-61.7485,78.80,20.71
2,6201424,Mataura,98,NZ,1612061858,35,-46.1927,168.8643,73.81,3.65
3,2126199,Cherskiy,0,RU,1612061856,83,68.7500,161.3000,-35.21,2.01
4,2173911,Broken Hill,34,AU,1612062402,28,-31.9500,141.4333,84.20,14.97
...,...,...,...,...,...,...,...,...,...,...
552,3868633,Vallenar,0,CL,1612062318,64,-28.5708,-70.7581,63.90,0.83
553,2192362,Christchurch,20,NZ,1612062280,63,-43.5333,172.6333,66.20,23.02
554,1052944,Angoche,100,MZ,1612062479,87,-16.2325,39.9086,77.14,7.20
555,2154826,Wonthaggi,36,AU,1612062479,63,-38.6044,145.5913,78.01,8.52


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=google_api)

In [4]:
#use lat&lng as locations for heat layer
locations = cities_df[['Lat','Lng']]
#use humidity to create the weights 
humid = cities_df['Humidity']

In [5]:
#setup your figure dimensions
figure_layout = {'width': '400px','height': '300px',
                 'border': '1px solid black',
                 'padding': '1px','margin': '0 auto 0 auto'}
#create figure using the layout from above
fig = gmaps.figure(layout=figure_layout)

# create heat layer and formatting
heat = gmaps.heatmap_layer(locations, weights=humid, dissipating=False, max_intensity=10, point_radius=1)

#add heat layer to map figure
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the DataFrame to find your ideal weather condition.

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.
  
* Drop any rows that don't contain all three conditions, or are null

In [6]:
# make a copy of df
ideal_df = cities_df.head()
ideal_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,1645133,Dumai,100,ID,1612062421,54,1.6833,101.4500,85.30,3.42
1,3579925,St. George's,20,GD,1612062421,83,12.0564,-61.7485,78.80,20.71
2,6201424,Mataura,98,NZ,1612061858,35,-46.1927,168.8643,73.81,3.65
3,2126199,Cherskiy,0,RU,1612061856,83,68.7500,161.3000,-35.21,2.01
4,2173911,Broken Hill,34,AU,1612062402,28,-31.9500,141.4333,84.20,14.97


In [7]:
#cities_df['Max_Temp']>100

In [8]:
# drop anything that does not meet all 3 cond

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
    * Set parameters to search for hotels with 5000 meters.
    * Hit the Google Places API for each city's coordinates.
    * Store the first Hotel result into the DataFrame.
    * Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**

In [16]:
# make a copy and add a new column for hotel name
hotel_df = ideal_df.copy()
hotel_df['Hotel_Name'] = ''
#create the base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#define the variables to use in our api call
params = {'type': 'lodging', 'radius':5000,'key':google_api,}
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel_Name
0,1645133,Dumai,100,ID,1612062421,54,1.6833,101.4500,85.30,3.42,
1,3579925,St. George's,20,GD,1612062421,83,12.0564,-61.7485,78.80,20.71,
2,6201424,Mataura,98,NZ,1612061858,35,-46.1927,168.8643,73.81,3.65,
3,2126199,Cherskiy,0,RU,1612061856,83,68.7500,161.3000,-35.21,2.01,
4,2173911,Broken Hill,34,AU,1612062402,28,-31.9500,141.4333,84.20,14.97,


In [17]:
for i, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat}, {lng}'
    response = rq.get(base_url,params=params).json()
    try:
        hotel_df.loc[i,'Hotel_Name'] = response['results'][0]['name']
    except:
        hotel_df.loc[i,'Hotel_Name'] = 'Hotel not in radius'
hotel_df        

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel_Name
0,1645133,Dumai,100,ID,1612062421,54,1.6833,101.4500,85.30,3.42,Grand Zuri Dumai
1,3579925,St. George's,20,GD,1612062421,83,12.0564,-61.7485,78.80,20.71,Siesta Hotel
2,6201424,Mataura,98,NZ,1612061858,35,-46.1927,168.8643,73.81,3.65,Ellie's Villa
3,2126199,Cherskiy,0,RU,1612061856,83,68.7500,161.3000,-35.21,2.01,Gostinitsa
4,2173911,Broken Hill,34,AU,1612062402,28,-31.9500,141.4333,84.20,14.97,Royal Exchange Hotel


In [11]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel_Name
0,1645133,Dumai,100,ID,1612062421,54,1.6833,101.4500,85.30,3.42,Royal Exchange Hotel
1,3579925,St. George's,20,GD,1612062421,83,12.0564,-61.7485,78.80,20.71,Royal Exchange Hotel
2,6201424,Mataura,98,NZ,1612061858,35,-46.1927,168.8643,73.81,3.65,Royal Exchange Hotel
3,2126199,Cherskiy,0,RU,1612061856,83,68.7500,161.3000,-35.21,2.01,Royal Exchange Hotel
4,2173911,Broken Hill,34,AU,1612062402,28,-31.9500,141.4333,84.20,14.97,Royal Exchange Hotel


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…